In [ ]:
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template
import pickle
import numpy as np


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2023-11-29 03:06:47--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 52.202.168.65, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  51.5MB/s    in 0.3s    

2023-11-29 03:06:48 (51.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# with open("/content/drive/MyDrive/project/model (1).pkl", 'rb') as file:
#     models = pickle.load(file)
#     print(models)

In [ ]:
with open('/content/drive/MyDrive/project/model1.pkl', 'rb') as file:
    model1 = pickle.load(file)

with open('/content/drive/MyDrive/project/model2.pkl', 'rb') as file:
    model2 = pickle.load(file)
with open('/content/drive/MyDrive/project/model3.pkl', 'rb') as file:
    model3 = pickle.load(file)
with open('/content/drive/MyDrive/project/model4.pkl', 'rb') as file:
    model4 = pickle.load(file)
with open('/content/drive/MyDrive/project/model5.pkl', 'rb') as file:
    model5 = pickle.load(file)



models = {
    "model1": model1,
    "model2": model2,
    "model3": model3,
    "model4": model4,
    "model5": model5
}

# Saving both models into a single pickle file
with open('combined_models.pkl', 'wb') as file:
    pickle.dump(models, file)

In [ ]:
print(models)

{'model1': <keras.src.engine.sequential.Sequential object at 0x7e5ae9e2e8f0>, 'model2': <keras.src.engine.sequential.Sequential object at 0x7e5ae8466ef0>, 'model3': <keras.src.engine.sequential.Sequential object at 0x7e5ae8382d40>, 'model4': <keras.src.engine.sequential.Sequential object at 0x7e5ae83bf1c0>, 'model5': <keras.src.engine.sequential.Sequential object at 0x7e5ae83e77c0>}


In [ ]:
!./ngrok authtoken 2YZvz2tb0harR9IzCS41noRUYLn_2ZQt6M86FuAcoZ2277c4W


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template
import pickle
import numpy as np

app = Flask(__name__, template_folder='/content/drive/MyDrive/project/template')
run_with_ngrok(app)  # Integrate Flask with ngrok to make the app accessible

# # Load multiple models from the pickle file
# with open("/content/drive/MyDrive/project/model (1).pkl", 'rb') as file:
#     models = pickle.load(file)

@app.route("/")
@app.route("/home")
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    model_name = request.form['company']  # Get the selected model name
    days = int(request.form['days'])

    open_val = float(request.form['open'])
    high_val = float(request.form['high'])
    low_val = float(request.form['low'])
    close_val = float(request.form['close'])

    # Convert values to a string separated by commas
    data_string = f"{open_val},{high_val},{low_val},{close_val}"

    # Split the string to get individual values
    values = data_string.split(",")

    # Convert string values to float
    numeric_values = [float(value) for value in values]

    # Reshape the data for prediction
    input_data = np.array(numeric_values).reshape(1, -1)

    # Get the selected model and make predictions
    if model_name in models:
      selected_model = models[model_name]

      predictions = []  # List to store predictions for each day

      for i in range(1, days + 1):  # Adjust range to include the last day
          prediction = selected_model.predict(input_data)

          predictions.append(prediction[0])  # Assuming prediction is a single value
          input_data = np.append(input_data[:, 1:], prediction).reshape(1, -1)

      # Format predictions for display
      prediction_text = "\n".join([f"Day {i}: {pred}" for i, pred in zip(range(1, days + 1), predictions)])
      prediction_text = f"Predicted outcomes for {days} days for {model_name}:\n{prediction_text}"

      return render_template('index.html', prediction=prediction_text)

    return render_template('index.html', prediction="Model not found")



if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://17a2-34-74-167-49.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [29/Nov/2023 03:07:46] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Nov/2023 03:07:47] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 0s 31ms/step


INFO:werkzeug:127.0.0.1 - - [29/Nov/2023 03:10:05] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 46ms/step


INFO:werkzeug:127.0.0.1 - - [29/Nov/2023 05:48:09] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 40ms/step


INFO:werkzeug:127.0.0.1 - - [29/Nov/2023 05:51:46] "POST /predict HTTP/1.1" 200 -
